# **DATA PREPROCESSING**

In [ ]:
import string
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
import os

drive.mount("/content/drive/")

In [ ]:
# Membaca data dari file CSV
data = pd.read_excel('/content/drive/MyDrive/Skripsi/data_skripsi/data_label_0_1.xlsx')
data.head()

In [ ]:
#Case Folding
data['textDisplay'] = data['textDisplay'].str.lower()
print('Case Folding Result : \n')
print(data['textDisplay'].head(5))
print('\n\n\n')
import string, re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

In [ ]:
data.head()

In [ ]:
#Tokenizing
def remove_comments_special(text):
    #menghapus karakter khusus dalam teks, seperti tab, newline, dan backslash
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\'," ").replace('.'," ").replace(','," ")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
data['textDisplay'] = data['textDisplay'].apply(remove_comments_special)

#menghapus angka dalam teks dengan menggunakan regular expression
def remove_number(text):
    return  re.sub(r"\d+", " ", text)
data['textDisplay'] = data['textDisplay'].apply(remove_number)
#menghapus emotikon dari teks menggunakan regular expression
def remove_emoticons(text):
    emoticon_pattern = re.compile("["
                                 u"\U0001F600-\U0001F64F"  # emoticons
                                 u"\U0001F300-\U0001F5FF"  # simbol & piktogram
                                 u"\U0001F680-\U0001F6FF"  # transportasi & simbol peralatan
                                 u"\U0001F1E0-\U0001F1FF"  # bendera negara
                                 u"\U00002702-\U000027B0"  # simbol lainnya
                                 u"\U000024C2-\U0001F251"
                                 "]+", flags=re.UNICODE)
    return emoticon_pattern.sub(r'', text)
data['textDisplay'] = data['textDisplay'].apply(remove_emoticons)
#data['review'] = data['review'].apply(remove_number)
def remove_punctuation(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator).replace(",", " ")
data['textDisplay'] = data['textDisplay'].apply(remove_punctuation)
# menghapus spasi yang ada di awal dan akhir teks menggunakan str.strip()
def remove_whitespace_LT(text):
    return text.strip()
data['textDisplay'] = data['textDisplay'].apply(remove_whitespace_LT)
#menghapus spasi berlebih dalam teks dengan menggantinya dengan satu spasi menggunakan regular expression
def remove_extra_spaces(text):
    return re.sub(r'\s+', ' ', text)
data['textDisplay'] = data['textDisplay'].apply(remove_extra_spaces)
#mengganti multiple whitespace (spasi berturut-turut) dengan satu spasi menggunakan regular expression
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)
data['textDisplay'] = data['textDisplay'].apply(remove_whitespace_multiple)
#menghapus huruf yang berulang dalam teks
def remove_repeated_letters(text):
    return re.sub(r"(.)\1+", r"\1", text)
data['textDisplay'] = data['textDisplay'].apply(remove_repeated_letters)

#menghapus kata-kata yang terdiri dari satu huruf (single character) dalam teks menggunakan regular expression
def filter_length(text):
    text_filtered = ' '.join(word for word in re.split(r'\s', text) if 1 < len(word))
    return text_filtered
data['textDisplay'] = data['textDisplay'].apply(filter_length)



#mengubah kalimat menjadi token
def word_tokenize_wrapper(text):
    return word_tokenize(text)
data['comments_tokens'] = data['textDisplay'].apply(word_tokenize_wrapper)
print('Tokenizing Result : \n')
print(data['comments_tokens'].head())
print('\n\n\n')

data.head()


In [ ]:
#Spelling Normalization
normalizad_word = pd.read_csv("https://raw.githubusercontent.com/meisaputri21/Indonesian-Twitter-Emotion-Dataset/master/kamus_singkatan.csv", sep=";", header=None)
normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1]

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]
data['comments_normalized'] = data['comments_tokens'].apply(normalized_term)

normalizad_word2 = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vRQS3tlUL5EcxYqbbYzFLHmHaqm2npjY-DLyz0dzwMIcUVhfoVWKuhR52P9YCqbAyY9zCgT66JVutWA/pub?output=csv",header=None)
normalizad_word_dict2 = {}

for index, row in normalizad_word2.iterrows():
    if row[0] not in normalizad_word_dict2:
        normalizad_word_dict2[row[0]] = row[1]
def normalized_term2(document):
    return [normalizad_word_dict2[term] if term in normalizad_word_dict2 else term for term in document]
data['comments_normalized'] = data['comments_normalized'].apply(normalized_term2)

In [ ]:
#menyatukan kembali tokenizer menjadi kalimat
def detokenize(token_list):
    # Menggabungkan token dengan spasi sebagai pemisah
    text = ' '.join(token_list)
    return text

data['comments_join'] = data['comments_normalized'].apply(detokenize)

print(data)

In [ ]:
# menghapus spasi yang ada di awal dan akhir teks menggunakan str.strip()
def remove_whitespace_LT(text):
    return text.strip()
data['comments_join'] = data['comments_join'].apply(remove_whitespace_LT)
#menghapus spasi berlebih dalam teks dengan menggantinya dengan satu spasi menggunakan regular expression
def remove_extra_spaces(text):
    return re.sub(r'\s+', ' ', text)
data['comments_join'] = data['comments_join'].apply(remove_extra_spaces)
#mengganti multiple whitespace (spasi berturut-turut) dengan satu spasi menggunakan regular expression
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)
data['comments_join'] = data['comments_join'].apply(remove_whitespace_multiple)

#mengubah kalimat menjadi token
def word_tokenize_wrapper(text):
    return word_tokenize(text)
data['comments_tokens_final'] = data['comments_join'].apply(word_tokenize_wrapper)
print('Tokenizing Result : \n')
print(data['comments_tokens_final'].head())
print('\n\n\n')


In [ ]:
#Stopword Removal
import pandas as pd

file_path = '/content/drive/MyDrive/Skripsi/data_skripsi/stop word indo (3).txt'  # Ganti dengan jalur file yang sesuai

# Baca isi file teks ke dalam list
with open(file_path, 'r') as file:
    lines = file.readlines()

# Mengubah setiap baris menjadi string dengan kutipan ganda
formatted_lines = ['"' + line.strip() + '"' for line in lines]

# Menggabungkan baris-baris menjadi satu string dengan koma sebagai pemisah
formatted_text = ", ".join(formatted_lines)

print(formatted_text)

# List kata stopwords
stopwords_list = formatted_text
def stopwords_removal(words):
    return [word for word in words if word not in stopwords_list]



data['comments_tokens_sw'] = data['comments_tokens_final'].apply(stopwords_removal)
data.head(100)


In [ ]:
print(data['comments_tokens_sw'].head())

data.head()


In [ ]:
#Stemming
# import Sastrawi package
! pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()
#stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)
term_dict = {}
for document in data['comments_tokens_sw']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
print(len(term_dict))
print("------------------------")
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
print(term_dict)
print("------------------------")
# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

data['comments_tokens_stemmed'] = data['comments_tokens_sw'].apply(get_stemmed_term)

# data["Ulasan_clean"] = [' '.join(map(str, l)) for l in data['comments_tokens_stemmed']]
# data.to_excel("Data_Skripsi_Bersih_.xlsx")

In [ ]:
#Stopword Removal
import pandas as pd

file_path = '/content/drive/MyDrive/Skripsi/data_skripsi/stop word indo (3).txt'  # Ganti dengan jalur file yang sesuai

# Baca isi file teks ke dalam list
with open(file_path, 'r') as file:
    lines = file.readlines()

# Mengubah setiap baris menjadi string dengan kutipan ganda
formatted_lines = ['"' + line.strip() + '"' for line in lines]

# Menggabungkan baris-baris menjadi satu string dengan koma sebagai pemisah
formatted_text = ", ".join(formatted_lines)

print(formatted_text)

# List kata stopwords
stopwords_list = formatted_text
def stopwords_removal(words):
    return [word for word in words if word not in stopwords_list]



data['comments_tokens_stemmed'] = data['comments_tokens_stemmed'].apply(stopwords_removal)
data.head(100)


In [ ]:
data["Ulasan_clean"] = [' '.join(map(str, l)) for l in data['comments_tokens_stemmed']]

In [ ]:
def filter_length(text):
    text_filtered = ' '.join(word for word in re.split(r'\s', text) if len(word) >= 4)
    return text_filtered

data['Ulasan_clean_fix'] = data['Ulasan_clean'].apply(filter_length)


In [ ]:
data.head()

In [ ]:
data.to_excel("Data_Skripsi_Bersih_Fix_.xlsx")

In [ ]:
###WordCloud
#Import Library untuk WordCloud
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud

text = " ".join(title for title in data["Ulasan_clean_fix"])
word_cloud = WordCloud(collocations = False, background_color = 'white',
                        width = 2048, height = 1080).generate(text)
#Menyimpan Gambar WordCloud
word_cloud.to_file('wordcloud.png')
#Menampilkan Hasil WordCloud
plt.imshow(word_cloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
#load data yang sudah bersih
df_preprocessed=pd.read_excel("/content/Data_Skripsi_Bersih_Fix_ .xlsx")
df_preprocessed

In [ ]:
# Menghapus baris yang tidak memiliki kata dalam kolom Ulasan_Clean dan memiliki nilai NaN
df_preprocessed = df_preprocessed.dropna(subset=['Ulasan_clean_fix'])

# Sekarang, Anda dapat melanjutkan dengan mengaplikasikan filter atau melakukan operasi lain pada DataFrame df_preprocessed

df_preprocessed.to_excel("df_preprocessed.xlsx")



In [ ]:
print(df_preprocessed.shape)

In [ ]:
df_preprocessed.Ulasan_clean=df_preprocessed.Ulasan_clean.astype(str)

**Word Embedding**

In [ ]:
import gzip
from urllib.request import urlopen
file = gzip.open(urlopen('https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.vec.gz'))

vocab_and_vectors = {}
# put words as dict indexes and vectors as words values
for line in file:
    values = line.split()
    word = values [0].decode('utf-8')
    vector = np.asarray(values[1:], dtype='float32')
    vocab_and_vectors[word] = vector

##pemberian indeks kata
df_preprocessed.Ulasan_clean=df_preprocessed.Ulasan_clean.astype(str)
# more imports
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# how many features should the tokenizer extract
features = 500
tokenizer = Tokenizer(num_words = features)
# fit the tokenizer on our text
texts = df_preprocessed.Ulasan_clean
tokenizer.fit_on_texts(texts)
# get all words that the tokenizer knows
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
# put the tokens in a matrix
X = tokenizer.texts_to_sequences(df_preprocessed["Ulasan_clean_fix"].tolist())
X = pad_sequences(X)
# prepare the labels
Y = df_preprocessed["label"]
from tensorflow.keras.utils import to_categorical
print(texts)

embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = vocab_and_vectors.get(word)
    # words that cannot be found will be set to 0
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Print sample of tokenized text
sample_tokenized = texts.sample(5)
for i, tokenized in sample_tokenized.items():
  print(tokenized)
  print(X[i])
  print('\n')

In [ ]:
# Print sample of word embeddings
sample_words = list(word_index.keys())[:5]  # Ambil beberapa kata dari indeks
for word in sample_words:
    word_index_value = word_index.get(word)
    if word_index_value is not None:
        embedding_vector = embedding_matrix[word_index_value]
        print(f"Word: {word}, Embedding Vector: {embedding_vector}")
    else:
        print(f"Word '{word}' not found in the vocabulary.")


**Split Data**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

#Pembagian Data
# Pembagian train (pelatihan) dan testing
Y = df_preprocessed["label"]
Y
print(Y.shape)
print(X.shape)

# One hot encoding label
import seaborn as sns
Y = to_categorical(Y, num_classes= 2)
Y
# split in train and test 80:20
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size= 0.2, stratify=Y, random_state=14)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)
# grafik data training dan testing
sns.countplot(Y_train,label='count')
plt.ylabel('Frekuensi', fontsize=12)
plt.xlabel('Label', fontsize=12)
plt.show()
sns.countplot(Y_test,label='count')
plt.ylabel('Frekuensi', fontsize=12)
plt.xlabel('Label', fontsize=12)
plt.show()


# **HYPERPARAMETER TUNING**

# **LEARNING RATE **

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import random

# Hyperparameters range
learning_rate_range = [0.0001, 0.001, 0.01, 0.1]

# Callbacks
early_stopping = EarlyStopping(patience=5, monitor='val_accuracy', mode='max', verbose=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model_1.h5', save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)

# Callbacks list
callbacks_list = [early_stopping, model_checkpoint]

#Build Model
def create_model(learning_rate):
    model = Sequential()
    model.add(Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False))
    model.add(Bidirectional(GRU(16)))
    model.add(Dropout(0.2))
    model.add(Dense(8, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(2, activation='sigmoid'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Tentukan jumlah percobaan yang ingin Anda lakukan
num_trials = 1

# Loop melalui setiap learning rate
for lr in learning_rate_range:
    for _ in range(num_trials):
        # Buat model baru untuk setiap iterasi
        model = create_model(lr)

        # Pelatihan model
        HistoryBiGRU = model.fit(X_train, Y_train, batch_size=64, epochs=100, validation_data=(X_test, Y_test), callbacks=callbacks_list)

        val_accuracy = HistoryBiGRU.history['val_accuracy'][-1]

        print(f"Trial {_ + 1} Hyperparameters:")
        print(f"Learning Rate: {lr}")
        print("\n")


# **Neurons**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import random

# Hyperparameters range
neurons = [8, 16,32, 64, 128]

# Randomly select hyperparameters

def create_model(neurons):
    model = Sequential()
    model.add(Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False))
    model.add(Bidirectional(GRU(neurons)))
    model.add(Dropout(0.5))
    model.add(Dense(8, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(2, activation='sigmoid'))

    optimizer = Adam(learning_rate=0.01)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Tentukan jumlah percobaan yang ingin Anda lakukan
num_trials = 1

# Tentukan nama file untuk menyimpan model terbaik
model_checkpoint = ModelCheckpoint('best_model_1.h5', save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)

# Gunakan EarlyStopping untuk menghentikan pelatihan saat tidak ada peningkatan dalam metrik yang dipantau
early_stopping = EarlyStopping(patience=5 ,monitor='val_accuracy', mode='max', verbose=1, restore_best_weights=True)

# Callbacks list yang berisi kedua callback
callbacks_list = [early_stopping, model_checkpoint]

# Callbacks list yang berisi kedua callback

for n in neurons:  # Loop melalui setiap dropout rate
    best_accuracy = 0
    best_model = None

    for _ in range(num_trials):
        # Di sini Anda perlu membuat model baru untuk setiap iterasi
        model = create_model(n)

        # Pelatihan model
        HistoryBiGRU = model.fit(X_train, Y_train, batch_size=64, epochs=100, validation_data=(X_test, Y_test), callbacks=callbacks_list)

        val_accuracy = HistoryBiGRU.history['val_accuracy'][-1]

        print(f"Trial {_ + 1} Hyperparameters:")
        print(f"Neurons: {n}")
        print("\n")


# **Dropout 1**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import random

# Hyperparameters range
dropout_rate = [0.2, 0.3, 0.4, 0.5, 0.6]

# Randomly select hyperparameters

def create_model(dropout):
    model = Sequential()
    model.add(Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False))
    model.add(Bidirectional(GRU(32)))
    model.add(Dropout(dropout))
    model.add(Dense(8, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(2, activation='sigmoid'))

    optimizer = Adam(learning_rate=0.01)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Tentukan jumlah percobaan yang ingin Anda lakukan
num_trials = 1

# Tentukan nama file untuk menyimpan model terbaik
model_checkpoint = ModelCheckpoint('best_model_1.h5', save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)

# Gunakan EarlyStopping untuk menghentikan pelatihan saat tidak ada peningkatan dalam metrik yang dipantau
early_stopping = EarlyStopping(patience=2, monitor='val_accuracy', mode='max', verbose=1, restore_best_weights=True)

# Callbacks list yang berisi kedua callback
callbacks_list = [early_stopping, model_checkpoint]

# Callbacks list yang berisi kedua callback

for do in dropout_rate:  # Loop melalui setiap dropout rate
    best_accuracy = 0
    best_model = None

    for _ in range(num_trials):
        # Di sini Anda perlu membuat model baru untuk setiap iterasi
        model = create_model(do)

        # Pelatihan model
        HistoryBiGRU = model.fit(X_train, Y_train, batch_size=64, epochs=10, validation_data=(X_test, Y_test), callbacks=callbacks_list)

        val_accuracy = HistoryBiGRU.history['val_accuracy'][-1]

        print(f"Trial {_ + 1} Hyperparameters:")
        print(f"Dropout 1: {do}")
        print("\n")


# **Dense Dim 1**


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import random

# Hyperparameters range
dense_dim = [8,16,32,64,128,512]

# Randomly select hyperparameters

def create_model(dense_1):
    model = Sequential()
    model.add(Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False))
    model.add(Bidirectional(GRU(32)))
    model.add(Dropout(0.5))
    model.add(Dense(dense_1, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(2, activation='sigmoid'))

    optimizer = Adam(learning_rate=0.01)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Tentukan jumlah percobaan yang ingin Anda lakukan
num_trials = 1

# Tentukan nama file untuk menyimpan model terbaik
model_checkpoint = ModelCheckpoint('best_model_1.h5', save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)

# Gunakan EarlyStopping untuk menghentikan pelatihan saat tidak ada peningkatan dalam metrik yang dipantau
early_stopping = EarlyStopping(patience=5, monitor='val_accuracy', mode='max', verbose=1, restore_best_weights=True)

# Callbacks list yang berisi kedua callback
callbacks_list = [early_stopping, model_checkpoint]

# Callbacks list yang berisi kedua callback

for den_1 in dense_dim:  # Loop melalui setiap dropout rate
    best_accuracy = 0
    best_model = None

    for _ in range(num_trials):
        # Di sini Anda perlu membuat model baru untuk setiap iterasi
        model = create_model(n)

        # Pelatihan model
        HistoryBiGRU = model.fit(X_train, Y_train, batch_size=64, epochs=100, validation_data=(X_test, Y_test), callbacks=callbacks_list)

        val_accuracy = HistoryBiGRU.history['val_accuracy'][-1]

        print(f"Trial {_ + 1} Hyperparameters:")
        print(f"dense_Dim_1: {den_1}")
        print("\n")


# **Dropoout_2**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import random

# Hyperparameters range
dropout_rate = [0.2, 0.3, 0.4, 0.5, 0.6]

# Randomly select hyperparameters

def create_model(dropout_2):
    model = Sequential()
    model.add(Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False))
    model.add(Bidirectional(GRU(32)))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(dropout_2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(2, activation='sigmoid'))

    optimizer = Adam(learning_rate=0.01)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Tentukan jumlah percobaan yang ingin Anda lakukan
num_trials = 1

# Tentukan nama file untuk menyimpan model terbaik
model_checkpoint = ModelCheckpoint('best_model_1.h5', save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)

# Gunakan EarlyStopping untuk menghentikan pelatihan saat tidak ada peningkatan dalam metrik yang dipantau
early_stopping = EarlyStopping(patience=5, monitor='val_accuracy', mode='max', verbose=1, restore_best_weights=True)

# Callbacks list yang berisi kedua callback
callbacks_list = [early_stopping, model_checkpoint]

# Callbacks list yang berisi kedua callback

for do_2 in dropout_rate:  # Loop melalui setiap dropout rate
    best_accuracy = 0
    best_model = None

    for _ in range(num_trials):
        # Di sini Anda perlu membuat model baru untuk setiap iterasi
        model = create_model(do)

        # Pelatihan model
        HistoryBiGRU = model.fit(X_train, Y_train, batch_size=64, epochs=100, validation_data=(X_test, Y_test), callbacks=callbacks_list)

        val_accuracy = HistoryBiGRU.history['val_accuracy'][-1]

        print(f"Trial {_ + 1} Hyperparameters:")
        print(f"Dropout 2: {do_2}")
        print("\n")


# **Dense Dim 2**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import random

# Hyperparameters range
dense_dim = [8,16,32,64,128,512]

# Randomly select hyperparameters

def create_model(dense_2):
    model = Sequential()
    model.add(Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False))
    model.add(Bidirectional(GRU(32)))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(dense_2, activation='relu'))
    model.add(Dense(2, activation='sigmoid'))

    optimizer = Adam(learning_rate=0.01)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Tentukan jumlah percobaan yang ingin Anda lakukan
num_trials = 1

# Tentukan nama file untuk menyimpan model terbaik
model_checkpoint = ModelCheckpoint('best_model_1.h5', save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)

# Gunakan EarlyStopping untuk menghentikan pelatihan saat tidak ada peningkatan dalam metrik yang dipantau
early_stopping = EarlyStopping(patience=5, monitor='val_accuracy', mode='max', verbose=1, restore_best_weights=True)

# Callbacks list yang berisi kedua callback
callbacks_list = [early_stopping, model_checkpoint]

# Callbacks list yang berisi kedua callback

for den_2 in dense_dim:  # Loop melalui setiap dropout rate
    best_accuracy = 0
    best_model = None

    for _ in range(num_trials):
        # Di sini Anda perlu membuat model baru untuk setiap iterasi
        model = create_model(n)

        # Pelatihan model
        HistoryBiGRU = model.fit(X_train, Y_train, batch_size=64, epochs=100, validation_data=(X_test, Y_test), callbacks=callbacks_list)

        val_accuracy = HistoryBiGRU.history['val_accuracy'][-1]

        print(f"Trial {_ + 1} Hyperparameters:")
        print(f"dense_Dim_2: {den_2}")
        print("\n")


# **batch size**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import random

# Hyperparameters range
batch_size = [8, 16, 32, 64]

# Randomly select hyperparameters

def create_model(batch_size):
    model = Sequential()
    model.add(Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False))
    model.add(Bidirectional(GRU(32)))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(2, activation='sigmoid'))
    optimizer = Adam(learning_rate=0.01)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Tentukan jumlah percobaan yang ingin Anda lakukan
num_trials = 1

# Tentukan nama file untuk menyimpan model terbaik
model_checkpoint = ModelCheckpoint('best_model_1.h5', save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)

# Gunakan EarlyStopping untuk menghentikan pelatihan saat tidak ada peningkatan dalam metrik yang dipantau
early_stopping = EarlyStopping(patience=5, monitor='val_accuracy', mode='max', verbose=1, restore_best_weights=True)

# Callbacks list yang berisi kedua callback
callbacks_list = [early_stopping, model_checkpoint]

# Callbacks list yang berisi kedua callback

for bs in batch_size:  # Loop melalui setiap dropout rate
    best_accuracy = 0
    best_model = None

    for _ in range(num_trials):
        # Di sini Anda perlu membuat model baru untuk setiap iterasi
        model = create_model(bs)

        # Pelatihan model
        HistoryBiGRU = model.fit(X_train, Y_train, batch_size=bs, epochs=100, validation_data=(X_test, Y_test), callbacks=callbacks_list)

        val_accuracy = HistoryBiGRU.history['val_accuracy'][-1]

        print(f"Trial {_ + 1} Hyperparameters:")
        print(f"batch_size: {bs}")
        print("\n")


**Model dengan Parameter Terpilih**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Dropout, BatchNormalization, Dense, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Definisi model di luar fungsi
model_2 = Sequential()
model_2.add(Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False))
model_2.add(Bidirectional(GRU(32)))
model_2.add(Dropout(0.5))
model_2.add(Dense(16, activation='relu'))
model_2.add(Dropout(0.4))
model_2.add(Dense(32, activation='relu'))
model_2.add(Dense(2, activation='sigmoid'))

optimizer = Adam(learning_rate=0.01)
model_2.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

def create_model_2():
    return model_2

# Tentukan nama file untuk menyimpan model_2 terbaik
model_2_checkpoint = ModelCheckpoint('best_model_6.h5', save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)

# Gunakan EarlyStopping untuk menghentikan pelatihan saat tidak ada peningkatan dalam metrik yang dipantau
early_stopping = EarlyStopping(patience=10, monitor='val_accuracy', mode='max', verbose=1, restore_best_weights=True)

# Callbacks list yang berisi kedua callback
callbacks_list = [early_stopping, model_2_checkpoint]

# Pemanggilan fungsi create_model_2() untuk mendapatkan model_2
model_2 = create_model_2()

# Pastikan Anda telah mendefinisikan X_train, Y_train, X_test, dan Y_test sebelumnya

HistoryBiGRU = model_2.fit(X_train, Y_train, batch_size=64, epochs=50, validation_data=(X_test, Y_test), callbacks=callbacks_list)

val_accuracy = HistoryBiGRU.history['val_accuracy'][-1]


In [ ]:
# from tensorflow.keras.models import save_model

# # Simpan model ke Google Drive
# model_path = '/content/drive/MyDrive/Skripsi/model_skripsi/best_model6/'
# save_model(best_model, model_path)

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/content/best_model_6.h5')


In [ ]:
import matplotlib.pyplot as plt

# Plot akurasi pelatihan dan validasi
plt.plot(HistoryBiGRU.history['accuracy'])
plt.plot(HistoryBiGRU.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot loss pelatihan dan validasi
plt.plot(HistoryBiGRU.history['loss'])
plt.plot(HistoryBiGRU.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


In [ ]:
# prediksi classes untuk data test
from sklearn.metrics import confusion_matrix, classification_report



y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis=1)
cm = confusion_matrix(y_true, y_pred)
print(cm)
print(classification_report(y_true, y_pred))
print(classification_report(y_true, y_pred, digits=4, output_dict=False))
report = classification_report(y_true, y_pred,
digits=4, output_dict=False)


In [ ]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(Y_test, axis=1)
# confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, y_pred_classes)
# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

# **KLASIFIKASI DATA BARU**





# **Data Preprocessing**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud

file_prediksi=pd.read_excel('/content/data_baru_klasifikasi.xlsx')
file_prediksi

In [ ]:
#Case Folding
file_prediksi['textDisplay'] = file_prediksi['textDisplay'].str.lower()
print('Case Folding Result : \n')
print(file_prediksi['textDisplay'].head(5))
print('\n\n\n')

In [ ]:
import string, re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

In [ ]:
#Tokenizing
def remove_comments_special(text):
    #menghapus karakter khusus dalam teks, seperti tab, newline, dan backslash
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\'," ").replace('.'," ").replace(','," ")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
file_prediksi['textDisplay'] = file_prediksi['textDisplay'].apply(remove_comments_special)

#menghapus angka dalam teks dengan menggunakan regular expression
def remove_number(text):
    return  re.sub(r"\d+", " ", text)
file_prediksi['textDisplay'] = file_prediksi['textDisplay'].apply(remove_number)
#menghapus emotikon dari teks menggunakan regular expression
def remove_emoticons(text):
    emoticon_pattern = re.compile("["
                                 u"\U0001F600-\U0001F64F"  # emoticons
                                 u"\U0001F300-\U0001F5FF"  # simbol & piktogram
                                 u"\U0001F680-\U0001F6FF"  # transportasi & simbol peralatan
                                 u"\U0001F1E0-\U0001F1FF"  # bendera negara
                                 u"\U00002702-\U000027B0"  # simbol lainnya
                                 u"\U000024C2-\U0001F251"
                                 "]+", flags=re.UNICODE)
    return emoticon_pattern.sub(r'', text)
file_prediksi['textDisplay'] = file_prediksi['textDisplay'].apply(remove_emoticons)
#file_prediksi['review'] = file_prediksi['review'].apply(remove_number)
def remove_punctuation(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator).replace(",", " ")
file_prediksi['textDisplay'] = file_prediksi['textDisplay'].apply(remove_punctuation)
# menghapus spasi yang ada di awal dan akhir teks menggunakan str.strip()
def remove_whitespace_LT(text):
    return text.strip()
file_prediksi['textDisplay'] = file_prediksi['textDisplay'].apply(remove_whitespace_LT)
#menghapus spasi berlebih dalam teks dengan menggantinya dengan satu spasi menggunakan regular expression
def remove_extra_spaces(text):
    return re.sub(r'\s+', ' ', text)
file_prediksi['textDisplay'] = file_prediksi['textDisplay'].apply(remove_extra_spaces)
#mengganti multiple whitespace (spasi berturut-turut) dengan satu spasi menggunakan regular expression
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)
file_prediksi['textDisplay'] = file_prediksi['textDisplay'].apply(remove_whitespace_multiple)
#menghapus huruf yang berulang dalam teks
def remove_repeated_letters(text):
    return re.sub(r"(.)\1+", r"\1", text)
file_prediksi['textDisplay'] = file_prediksi['textDisplay'].apply(remove_repeated_letters)

def filter_length(text):
    # Menghapus kata-kata yang kurang dari 4 karakter atau lebih dari 20 karakter
    text_filtered = ' '.join(word for word in re.split(r'\s', text) if 3 < len(word) < 26)
    return text_filtered
file_prediksi['textDisplay'] = file_prediksi['textDisplay'].apply(filter_length)

#mengubah textDisplay menjadi token
def word_tokenize_wrapper(text):
    return word_tokenize(text)
file_prediksi['comments_tokens'] = file_prediksi['textDisplay'].apply(word_tokenize_wrapper)
print('Tokenizing Result : \n')
print(file_prediksi['comments_tokens'].head())
print('\n\n\n')


In [ ]:
file_prediksi

In [ ]:
#Spelling Normalization
normalizad_word = pd.read_csv("https://raw.githubusercontent.com/meisaputri21/Indonesian-Twitter-Emotion-Dataset/master/kamus_singkatan.csv", sep=";", header=None)
normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1]

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]
file_prediksi['comments_normalized'] = file_prediksi['comments_tokens'].apply(normalized_term)

normalizad_word2 = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vRQS3tlUL5EcxYqbbYzFLHmHaqm2npjY-DLyz0dzwMIcUVhfoVWKuhR52P9YCqbAyY9zCgT66JVutWA/pub?output=csv",header=None)
normalizad_word_dict2 = {}

for index, row in normalizad_word2.iterrows():
    if row[0] not in normalizad_word_dict2:
        normalizad_word_dict2[row[0]] = row[1]
def normalized_term2(document):
    return [normalizad_word_dict2[term] if term in normalizad_word_dict2 else term for term in document]
file_prediksi['comments_normalized'] = file_prediksi['comments_normalized'].apply(normalized_term2)

In [ ]:
#menyatukan kembali tokenizer menjadi kalimat
def detokenize(token_list):
    # Menggabungkan token dengan spasi sebagai pemisah
    text = ' '.join(token_list)
    return text

file_prediksi['comments_join'] = file_prediksi['comments_normalized'].apply(detokenize)

print(file_prediksi)

In [ ]:
# menghapus spasi yang ada di awal dan akhir teks menggunakan str.strip()
def remove_whitespace_LT(text):
    return text.strip()
file_prediksi['comments_join'] = file_prediksi['comments_join'].apply(remove_whitespace_LT)
#menghapus spasi berlebih dalam teks dengan menggantinya dengan satu spasi menggunakan regular expression
def remove_extra_spaces(text):
    return re.sub(r'\s+', ' ', text)
file_prediksi['comments_join'] = file_prediksi['comments_join'].apply(remove_extra_spaces)
#mengganti multiple whitespace (spasi berturut-turut) dengan satu spasi menggunakan regular expression
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)
file_prediksi['comments_join'] = file_prediksi['comments_join'].apply(remove_whitespace_multiple)

#mengubah kalimat menjadi token
def word_tokenize_wrapper(text):
    return word_tokenize(text)
file_prediksi['comments_tokens_final'] = file_prediksi['comments_join'].apply(word_tokenize_wrapper)
print('Tokenizing Result : \n')
print(file_prediksi['comments_tokens_final'].head())
print('\n\n\n')


In [ ]:
#Stopword Removal
import pandas as pd

file_path = '/content/drive/MyDrive/Skripsi/data_skripsi/stop word indo (3).txt'  # Ganti dengan jalur file yang sesuai

# Baca isi file teks ke dalam list
with open(file_path, 'r') as file:
    lines = file.readlines()

# Mengubah setiap baris menjadi string dengan kutipan ganda
formatted_lines = ['"' + line.strip() + '"' for line in lines]

# Menggabungkan baris-baris menjadi satu string dengan koma sebagai pemisah
formatted_text = ", ".join(formatted_lines)

print(formatted_text)

# List kata stopwords
stopwords_list = formatted_text
def stopwords_removal(words):
    return [word for word in words if word not in stopwords_list]


file_prediksi['comments_tokens_final'] = file_prediksi['comments_tokens_final'].apply(stopwords_removal)
file_prediksi.head(100)

In [ ]:
# file_prediksi.to_excel('/content/prediksi.xlsx', index=False)


In [ ]:
#Stemming
# import Sastrawi package
! pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()
#stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)
term_dict = {}
for document in file_prediksi['comments_tokens_final']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
print(len(term_dict))
print("------------------------")
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
print(term_dict)
print("------------------------")
# apply stemmed term to file_prediksiframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

file_prediksi['comments_tokens_stemmed'] = file_prediksi['comments_tokens_final'].apply(get_stemmed_term)

# file_prediksi["Ulasan_clean"] = [' '.join(map(str, l)) for l in file_prediksi['comments_tokens_stemmed']]



In [ ]:
#Stopword Removal
import pandas as pd

file_path = '/content/drive/MyDrive/Skripsi/data_skripsi/stop word indo (3).txt'  # Ganti dengan jalur file yang sesuai

# Baca isi file teks ke dalam list
with open(file_path, 'r') as file:
    lines = file.readlines()

# Mengubah setiap baris menjadi string dengan kutipan ganda
formatted_lines = ['"' + line.strip() + '"' for line in lines]

# Menggabungkan baris-baris menjadi satu string dengan koma sebagai pemisah
formatted_text = ", ".join(formatted_lines)

print(formatted_text)

# List kata stopwords
stopwords_list = formatted_text
def stopwords_removal(words):
    return [word for word in words if word not in stopwords_list]


file_prediksi['comments_tokens_stemmed'] = file_prediksi['comments_tokens_stemmed'].apply(stopwords_removal)
file_prediksi.head(100)

In [ ]:
file_prediksi["Ulasan_clean"] = [' '.join(map(str, l)) for l in file_prediksi['comments_tokens_stemmed']]

In [ ]:
def remove_rows_without_words(row):
    return len(row['Ulasan_clean'].split()) > 0  # Jika jumlah kata lebih dari 0, baris tetap

# Menghapus baris yang tidak memiliki kata dalam kolom Ulasan_Clean
file_prediksi = file_prediksi[file_prediksi.apply(remove_rows_without_words, axis=1)]
# Hapus baris dengan kolom 'Ulasan_clean' tanpa nilai (empty)
file_prediksi = file_prediksi.dropna(subset=['Ulasan_clean'])


In [ ]:
file_prediksi.head()

In [ ]:
# Count the number of words in each list of 'comments_tokens_sw'
file_prediksi['word_count'] = file_prediksi['comments_tokens_stemmed'].apply(lambda x: len(x))

# Filter rows where the word count is greater than or equal to 5
file_prediksi = file_prediksi[file_prediksi['word_count'] >= 4]

# Drop the 'word_count' column if you don't need it anymore
file_prediksi = file_prediksi.drop(columns=['word_count'])

# Print the resulting DataFrame
print(file_prediksi.head())
file_prediksi.to_excel("Data_Skripsi_Bersih_prediksi_databaru.xlsx")


In [ ]:
print(file_prediksi.shape)

**Klasifikasi Data Dengan Model Bi-GRU**

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the Excel file
file_path = '/content/Data_Skripsi_Bersih_prediksi_databaru.xlsx'
data = pd.read_excel(file_path)

# # Menghapus baris dengan nilai NaN pada kolom 'Ulasan_Clean'
# data = data.dropna(subset=['Ulasan_clean'])

# Preprocess and predict for each sentence in the Excel file
predicted_sentiments = []

for sentence in data['Ulasan_clean']:
    # Pre-processing steps like tokenization, normalization, removing stopwords, stemming, etc.
    new_sequence = tokenizer.texts_to_sequences([sentence])
    new_sequence = pad_sequences(new_sequence, maxlen=100)

    # Perform prediction using the model
    new_prediction = model.predict(new_sequence)
    predicted_class = np.argmax(new_prediction)

    if predicted_class == 0:
        predicted_sentiments.append("Sentimen Negatif")
    else:
        predicted_sentiments.append("Sentimen Positif")

# Add the predicted sentiments to the dataFrame
data['Prediksi Sentimen'] = predicted_sentiments

# Save the dataFrame with predictions back to the Excel file
data.to_excel(file_path, index=False)


# **Topic Modelling**

In [ ]:
pip install --upgrade gensim

In [ ]:
pip install pyldavis==3.2.1

In [ ]:
pip install swifter

**Preparing Data**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud

data=pd.read_excel('/content/Data_Skripsi_Bersih_prediksi_databaru.xlsx')
data

In [ ]:
# Filter rows where the word count is greater than or equal to 5
data['word_count'] = data['Ulasan_clean'].apply(lambda x: len(x.split()))
data = data[data['word_count'] > 4]



# **LDA MODEL USING GENSIM**

In [ ]:
import gensim
from gensim import corpora

# Filter rows where the sentiment is negative
negative_sentiment_data = data[data['Prediksi Sentimen'] == 'Sentimen Negatif']


# Print the resulting DataFrame
print(negative_sentiment_data.head())
doc_clean = negative_sentiment_data["Ulasan_clean"].apply(lambda x: x.split())

dictionary = corpora.Dictionary(doc_clean)
print(dictionary)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [ ]:
negative_sentiment_data.shape

In [ ]:
###WordCloud
#Import Library untuk WordCloud
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud

text = " ".join(title for title in negative_sentiment_data["Ulasan_clean"])
word_cloud = WordCloud(collocations = False, background_color = 'white',
                        width = 2048, height = 1080).generate(text)
#Menyimpan Gambar WordCloud
word_cloud.to_file('wordcloud.png')
#Menampilkan Hasil WordCloud
plt.imshow(word_cloud, interpolation='bilinear')
plt.axis("off")
plt.show()

# **HYPERPARAMETER TOPIK**

In [ ]:
from gensim.models import CoherenceModel, LdaModel


# Assuming you have already defined doc_clean, doc_term_matrix, and dictionary

corpus = [dictionary.doc2bow(doc) for doc in doc_clean]

def evaluate_model(model, corpus, texts, num_topics):
    # Compute coherence score
    coherence_model = CoherenceModel(model=model, texts=doc_clean, corpus=corpus, coherence='c_v')
    coherence_score = coherence_model.get_coherence()

    return coherence_score

# Running and Training LDA model on the document term matrix.
total_topics = [2, 3, 4, 5, 6, 7]  # Or other values as needed

highest_coherence_score = 0
best_model = None

for num_topics in total_topics:
    # Initialize model with a specific number of topics
    model = LdaModel(doc_term_matrix, num_topics=num_topics, id2word=dictionary, passes=10)


    # Evaluate model
    coherence_score = evaluate_model(model, corpus, doc_clean, num_topics)

    # Print results
    print(f"Num Topics: {num_topics}, Coherence Score: {coherence_score}")

    # Check if the current model has a higher coherence score
    if coherence_score > highest_coherence_score:
        highest_coherence_score = coherence_score
        best_model = model

# Save the best model
best_model.save("best_lda_model")

# Print information about the best model
print(f"Best Model - Num Topics: {best_model.num_topics}, Coherence Score: {highest_coherence_score}")


In [ ]:
from gensim.models import LdaModel
best_model = LdaModel.load("/content/best_lda_model")

In [ ]:
def evaluate_model(model, corpus, texts, num_topics):
    # Compute coherence score
    coherence_model = CoherenceModel(model=best_model, texts=texts, corpus=corpus, coherence='c_v')
    coherence_score = coherence_model.get_coherence()

    # Print coherence score
    print(f"Coherence Score for {num_topics} topics: {coherence_score}")

    # Print beta values for each topic
    for topic_id in range(num_topics):
        topic_terms = best_model.get_topic_terms(topicid=topic_id, topn=10)  # Adjust topn as needed
        print(f"Topik {topic_id}:")
        for term_id, beta in topic_terms:
            print(f"- Kata: {dictionary[term_id]}, Nilai Beta: {beta:.2f}")  # Format untuk 2 angka di belakang koma

    return coherence_score

# Print information about the best model

print("Beta values for best model:")
evaluate_model(best_model, corpus, doc_clean, best_model.num_topics)

# **HYPERPARAMETER ALPHA DAN BETA**

In [ ]:
# @title Default title text
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary

# Define a function to evaluate model with specific alpha and beta
def evaluate_model_with_hyperparams(corpus, texts, num_topics, alpha, beta):
    model = LdaModel(corpus=corpus, num_topics=4, id2word=dictionary, passes=10, alpha=alpha, eta=beta)
    coherence_model = CoherenceModel(model=model, texts=texts, corpus=corpus, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    return coherence_score, model

# Running and Training LDA model on the document term matrix.
total_alpha_values = [0.01,0.1,1,'symmetric', 'asymmetric']  # Alpha values to try
total_beta_values = [0.01,0.1,1,'symmetric']   # Beta values to try
best_coherence_score = 0
best_alpha = None
best_beta = None
best_model = None

for alpha in total_alpha_values:
    for beta in total_beta_values:
        # Evaluate model with specific alpha and beta
        coherence_score, model = evaluate_model_with_hyperparams(corpus, doc_clean, num_topics=4, alpha=alpha, beta=beta)

        # Print results
        print(f"Alpha: {alpha}, Beta: {beta}, Coherence Score: {coherence_score}")

        # Check if the current model has a higher coherence score
        if coherence_score > best_coherence_score:
            best_coherence_score = coherence_score
            best_alpha = alpha
            best_beta = beta
            best_model = model

# Print the best hyperparameters
print(f"Best Alpha: {best_alpha}, Best Beta: {best_beta}, Best Coherence Score: {best_coherence_score}")

# Save the best model
best_model.save("best_lda_model_hyperparameter")


In [ ]:
from gensim.models import LdaModel
best_model_hyper = LdaModel.load("/content/best_lda_model_hyperparameter")

In [ ]:
# Check alpha
alpha = best_model.alpha

# Check beta
beta = best_model.eta

print("Alpha:", alpha)
print("Beta:", beta)

In [ ]:
from gensim.models import CoherenceModel

# Menghitung skor koherensi menggunakan CoherenceModel
coherence_model = CoherenceModel(model=best_model_hyper, texts=doc_clean, coherence='c_v')
coherence_score = coherence_model.get_coherence()

# Print skor koherensi
print("Coherence Score:", coherence_score)


In [ ]:
def evaluate_model(model, corpus, texts, num_topics):
    # Compute coherence score
    coherence_model = CoherenceModel(model=best_model, texts=texts, corpus=corpus, coherence='c_v')
    coherence_score = coherence_model.get_coherence()

    # Print coherence score
    print(f"Coherence Score for {num_topics} topics: {coherence_score}")

    # Print beta values for each topic
    for topic_id in range(num_topics):
        topic_terms = best_model.get_topic_terms(topicid=topic_id, topn=10)  # Adjust topn as needed
        print(f"Topik {topic_id}:")
        for term_id, beta in topic_terms:
            print(f"- Kata: {dictionary[term_id]}, Nilai Beta: {beta:.2f}")  # Format untuk 2 angka di belakang koma

    return coherence_score

# Print information about the best model

print("Beta values for best model:")
evaluate_model(best_model, corpus, doc_clean, best_model.num_topics)

In [ ]:
doc_clean.to_excel('nama_file1.xlsx', index=False)

In [ ]:
# Word Count of Topic Keywords

from collections import Counter
topics = best_model.show_topics(formatted=False)
data_flat = [w for w_list in doc_clean for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df_imp_wcount = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])
print(df_imp_wcount)

In [ ]:
#Dominant topic and its percentage contribution in each topic
def format_topics_sentences(ldamodel=None, corpus=doc_term_matrix, texts=doc_clean):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [ ]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=best_model, corpus=doc_term_matrix, texts=doc_clean)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
print(df_dominant_topic.head(10))

In [ ]:
import pyLDAvis.gensim
import pickle
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()

In [ ]:
import os
LDAvis_data_filepath = os.path.join('ldavis_prepared_'+str(4))

In [ ]:
corpus = [dictionary.doc2bow(text) for text in doc_clean]

In [ ]:
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(best_model, corpus, dictionary)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

In [ ]:
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

In [ ]:
LDAvis_prepared

In [ ]:
# Mencetak nama-nama kolom
print(df_topic_sents_keywords.columns)

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Contoh: Membuat word cloud untuk setiap topik
for topic_id in df_imp_wcount['topic_id'].unique():
    # Filter DataFrame untuk mendapatkan data terkait topik tertentu
    topic_data = df_imp_wcount[df_imp_wcount['topic_id'] == topic_id]

    # Menggabungkan kata-kata kunci dari semua kata dalam topik tersebut
    topic_keywords = ' '.join(topic_data['word'])

    # Membuat word cloud
    wordcloud = WordCloud(width=800, height=400, background_color="white").generate(topic_keywords)

    # Menampilkan word cloud
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"Word Cloud for Topic {topic_id}")
    plt.show()
